In [50]:
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv
import _pickle as cPickle
import os
import numpy as np
from mmdet.visualization.palette import get_palette
from mmdet.visualization.utils.data_utils import get_K,get_bbox_conner,reproj_3d_2d
import cv2

In [51]:
# download the checkpoint demo
#!mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest ./checkpoints
# config_file = '/root/userfolder/github/mmdetection/work_dirs/dab-detr_nocs_ViTAdapter_IOU_CRtrain_3enc_3dec_detachTS_expandgt/dab-detr_nocs_ViTAdapter_IOU_CRtrain_3enc_3dec.py'
# checkpoint_file = '/root/userfolder/github/mmdetection/work_dirs/dab-detr_nocs_ViTAdapter_IOU_CRtrain_3enc_3dec_detachTS_expandgt/epoch_10.pth'
config_file = '/root/userfolder/github/mmdetection/work_dirs/dab-detr_nocs_ViTDINO_Norm_CRtrain_3enc_3dec_2.12_8GPUS/dab-detr_nocs_ViTDINO_Norm_CRtrain_3enc_3dec.py'
checkpoint_file = '/root/userfolder/github/mmdetection/work_dirs/dab-detr_nocs_ViTDINO_Norm_CRtrain_3enc_3dec_2.12_8GPUS/epoch_48.pth'
# config_file = '/root/userfolder/github/mmdetection/work_dirs/dab-detr_sunrgbd_ViTDET_3enc_3dec_no_scale_1024_sam_preprocess_random_aug/dab-detr_sunrgbd_ViTDET_3enc_3dec_no_scale_1024_sam_preprocess_random_aug.py'
# checkpoint_file = '/root/userfolder/github/mmdetection/work_dirs/dab-detr_sunrgbd_ViTDET_3enc_3dec_no_scale_1024_sam_preprocess_random_aug/epoch_800.pth'

In [52]:
#Register all modules in mmdet into the registries
register_all_modules()
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cpu')  # or device='cpu' 'device:0'

Loads checkpoint by local backend from path: /root/userfolder/github/mmdetection/work_dirs/dab-detr_nocs_ViTDINO_Norm_CRtrain_3enc_3dec_2.12_8GPUS/epoch_48.pth


In [53]:
# data_root = '/root/commonfile/fxf/nocs/Real/'
# file = 'test_list.txt'
# visual_list = []
# out_file_list = []
# gt_result_list = []
# img_list_file = open(os.path.join(data_root,file),'r')
# img_list = img_list_file.read().splitlines()
# for img_dir in img_list:
#     if img_dir.endswith('00') or img_dir.endswith('50'):
#         visual_list.append(os.path.join(data_root,img_dir)+"_color.png")
#         out_file_list.append(img_dir.replace('/','')+"vis.png")
#         gt_result_list.append(os.path.join(data_root,img_dir)+"_label.pkl")
# print(visual_list)
# print(out_file_list)
# print(gt_result_list)


data_root = '/root/commonfile/fxf/nocs/CAMERA/'
file = 'val_list.txt'
visual_list = []
out_file_list = []
gt_result_list = []
img_list_file = open(os.path.join(data_root,file),'r')
img_list = img_list_file.read().splitlines()
for img_dir in img_list:
    if img_dir.endswith('00/0001') or img_dir.endswith('00/0005') or img_dir.endswith('05/0003'):
        visual_list.append(os.path.join(data_root,img_dir)+"_color.png")
        out_file_list.append(img_dir.replace('/','')+"vis.png")
        gt_result_list.append(os.path.join(data_root,img_dir)+"_label.pkl")
print(visual_list)
print(out_file_list)
print(gt_result_list)


['/root/commonfile/fxf/nocs/CAMERA/val/00000/0001_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00000/0005_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00005/0003_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00100/0001_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00100/0005_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00105/0003_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00200/0005_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00205/0003_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00300/0005_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00305/0003_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00400/0001_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00400/0005_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00405/0003_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00500/0001_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00500/0005_color.png', '/root/commonfile/fxf/nocs/CAMERA/val/00600/0001_color.png', '/root/commonfile/fxf/n

In [54]:
# test a single image
#img = mmcv.imread( '/root/commonfile/fxf/nocs/Real/train/scene_2/0000_color.png', channel_order='rgb')
#img = mmcv.imread( '/root/commonfile/fxf/nocs/CAMERA/val/00007/0002_color.png', channel_order='rgb')
#img = mmcv.imread( '/root/commonfile/fxf/nocs/Real/test/scene_1/0000_color.png', channel_order='rgb')
#img = mmcv.imread( '/root/commonfile/fxf/sunrgbd/sunrgbd_trainval/image/008000.jpg', channel_order='rgb')
img_list=[]
result_list=[]
for visual_dir,gt_result_dir in zip(visual_list,gt_result_list):
    img = cv2.imread( visual_dir)
    with open (gt_result_dir,'rb') as f:
        gt_labels=cPickle.load(f)
    img_list.append(img)
    result_list.append(gt_labels)
print(gt_labels)
    

{'class_ids': array([2, 5, 1]), 'bboxes': array([[316, 101, 389, 195],
       [ 78, 300, 182, 443],
       [147, 305, 243, 348]], dtype=int32), 'instance_ids': [1, 2, 3], 'model_list': ['fa23aa60ec51c8e4c40fe5637f0a27e1', 'ed35ed3a7c8824e5996bace9c8deb982', 'de71aa5eca9ee76a95f577622f465c85'], 'size': array([[0.67827987, 0.271874  , 0.67331217],
       [0.516162  , 0.49778965, 0.696674  ],
       [0.38047369, 0.84947   , 0.36217291]]), 'scales': array([0.2020867 , 0.40790707, 0.236136  ], dtype=float32), 'rotations': array([[[-0.86041325, -0.00431592, -0.50957865],
        [ 0.3146541 , -0.79107594, -0.5245871 ],
        [-0.40085134, -0.6117027 ,  0.6820103 ]],

       [[-0.51387316, -0.00310924,  0.85786057],
        [-0.52443564, -0.7902353 , -0.31701007],
        [ 0.6788974 , -0.6127956 ,  0.4044501 ]],

       [[-0.4467234 , -0.0045775 ,  0.8946604 ],
        [-0.5456057 , -0.7911213 , -0.27648056],
        [ 0.70905054, -0.6116421 ,  0.3509149 ]]], dtype=float32), 'translations'

In [55]:
# # init the visualizer(execute this block only once)
# visualizer = VISUALIZERS.build(model.cfg.visualizer)
# # the dataset_meta is loaded from the checkpoint and
# # then pass to the model in init_detector
palette = model.dataset_meta.get('palette', None)
classes = model.dataset_meta.get('classes', None)

In [56]:
# show the results
for img,result,out_file in zip(img_list,result_list,out_file_list):
    # print(img)
    # print(result)
    # visualizer.add_datasample(
    #     'result',
    #     img,
    #     data_sample=result,
    #     draw_gt=False,
    #     wait_time=0,
    #     pred_score_thr=0.2,
    #     out_file=out_file,
    # )
    img_result=img
    nocs_camera_intrinsics = [577.5, 577.5, 319.5, 239.5]
    nocs_real_intrinsics = [591.0125, 590.16775, 322.525, 244.11084]
    fx, fy, cx, cy = nocs_camera_intrinsics
    fx, fy, cx, cy = nocs_real_intrinsics
    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0,  0,  1]
    ])
    rotations = result['rotations']
    scales = result['scales'].reshape(-1,1)
    positions = result['translations']
    labels = result['class_ids']
    sizes = result['size']
    sizes = np.multiply(sizes,scales)
    bboxes = result['bboxes']

    bbox_color = palette
    max_label = int(max(labels) if len(labels) > 0 else 0)
    bbox_palette = get_palette(bbox_color, max_label + 10)

    for rot,pos,label,size,bbox in zip (rotations,positions,labels,sizes,bboxes):
        # print(rot,pos,label)
        label=label-1
        label_text = classes[label]
        colors = bbox_palette[label]
        #bbox_size=class_annotations[str(label.item())]["scales"]['0'] #获取bbox的大小从而获取角点位置
        bbox_size=size
        # x,y,z=size #预测的bboox size更加准确
        # bbox_size=[z,y,x] #for omni3d
        bbox_conner=get_bbox_conner(bbox_size)
        pose_R = rot.reshape(3,3)
        pose_T = pos.reshape(3,1)
        pose_RT=np.concatenate([pose_R,pose_T],axis=-1)
        reproj_box_2d = reproj_3d_2d(K, pose_RT, bbox_conner)
        # print(reproj_box_2d)
        # print('draw_3d_box')
        lines = np.array([
            [0, 1, 5, 4, 2, 3, 7, 6, 2, 2, 3, 7],
            [1, 5, 4, 0, 3, 0, 6, 5, 1, 6, 7, 4]
        ]).T

        font=cv2.FONT_HERSHEY_SIMPLEX
        font_scale=0.55
        thickness = 1
        retval, baseLine =cv2.getTextSize(label_text, font, font_scale, thickness)
        topleft = (bbox[1]-15, bbox[0] - retval[1]-5)
        bottomright = (topleft[0] + retval[0], topleft[1] + retval[1])

        cv2.rectangle(img_result, (topleft[0], topleft[1] - baseLine), bottomright,thickness=-1, color=(0,0,0))
        
        for id, line in enumerate(lines):
            pt1 = reproj_box_2d[line[0]].astype(int)
            pt2 = reproj_box_2d[line[1]].astype(int)
            cv2.line(img_result, tuple(pt1), tuple(pt2), colors, 2)

        cv2.putText(img_result,label_text,org=(bbox[1]-15,bbox[0]-10),
                    fontFace=font,fontScale=font_scale,color=(224,224,224),thickness=thickness)
        
    cv2.imwrite(filename=out_file,img=img_result)
                


